# I- LIBRARY INSTALLS

In [ ]:
# The PyTorch package is officially named torch on pip

# ! pip install transformers datasets torch 
# ! conda install pytorch torchvision torchaudio pytorch-cuda=12.4 -c pytorch -c nvidia 

# ! pip show torch
# ! conda install faiss-gpu 

# ! conda install -c conda-forge datasets --update-deps
# ! pip install --upgrade datasets

# !pip install pyarrow

# beautifulsoup4 is the name to install via pip
# !pip install beautifulsoup4 

II- LIBRARY IMPORTS

In [ ]:
%%time
import os
import gzip
import itertools
import json
import h5py
#import tables # After conda install -c anaconda pytables
import gc
#import zipfile

import requests
import numpy as np
import pandas as pd
import pickle


import time
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# bs4 is the module of beautifulsoup4 that contains the BeautifulSoup class.
from bs4 import BeautifulSoup

#import pyarrow as pa
#import pyarrow.feather as feather
#import pyarrow.parquet as pq
#import pyarrow.ipc as ipc

# Hugging Face's Transformers library primarily supports RAG models in PyTorch
import torch, torchvision, torchaudio
import torch.nn.functional as F

#  FAISS MEANS "Facebook AI Similarity Search"
# TO CRREATE INDECES ON TEXT KNOWLED BASE
import faiss

# DPR MEANS Dense Passage Retrieval (ENCODE QUERY WITH KNOWLEDGE BASE)
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder, RagRetriever, RagTokenizer, RagTokenForGeneration
from transformers import BartTokenizerFast, DPRQuestionEncoderTokenizerFast
from datasets import load_dataset, Dataset


os.getcwd()


CPU times: user 1.78 s, sys: 260 ms, total: 2.04 s
Wall time: 4.38 s


'/mnt/d/ANALYTICS - Continuous Education/RAG'

### III-A.1. CREATE AS "TAB-SAPARATED-VALUE (TSV)" FILE THE TEXT PASSAGES (AKA KNOWLEDGE BASE)

In [ ]:
# GENERATOR FUNCTION FOR "CHUNK-LOADING" DATA FROM A FILE (HERE ZIPPED)
# USING "itertools.islice"

def chunks_generator_for_gz_zipped_file(data_path, chunk_size):

    with gzip.open(data_path, 'rt', encoding='utf-8') as f:

        # READ ONLY A NUMBER OF ROWS FROM THE SAVED-TO-DISK DATASET THAT IS EQUAL TO "chunk_size"
        #chunk_count = 0 
        for chunk in iter(lambda: list(itertools.islice(f, chunk_size)), []): 
             
            # LIMIT THE NUMBER OF CHUNKS TO READ FROM THE SAVED-TO-DISK DATASET
            #if chunk_count >= 10:  
            #    break            
            
            # YIELD ONLY EXIST WITHIN A FUNCTION
            yield chunk
            #chunk_count += 1 

In [ ]:
# FUNCTION TO EXTRACT "TEXT PASAGES" FROM AN  HTML STRING ALREADY EXISTING IN THE DATASET

def extract_text_from_wikipedia_html(document_text, row_number, modulo_for_print):    
    
    paragraphs_as_content_div_find_all_p_is_empty={}
    
    #print(document_text)
    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(document_text, 'html.parser')
    content_div = soup.find_all('p')     
    #print("content_div", content_div, type(content_div))
    
    if content_div is None:                  
          print("Content section 'mw-content-text' is None for Row number:", row_number)          
          text_content = " "
          return text_content        

    cleaned_paragraphs = []

     # Loop through the content and extract text from paragraphs
    for element in content_div:
          #text = element.get_text(strip=True)
          text = element.get_text(strip=False)                          
          #print(text)
          # Only append non-empty text
          if len(text.strip()) > 0:                 
               cleaned_paragraphs.append(text)    

     # Combine all paragraphs into a single string
    text_content = ' '.join(cleaned_paragraphs)
     # Replace newline characters with a space
    text_content = text_content.replace("\n", " ").strip()
     #text_content = text_content.replace("\t", " ")

    if not len(text_content.strip()) >0:
                              
          paragraphs_as_content_div_find_all_p_is_empty[row_number]=( content_div)
          #print(row_number,":", url, "\n", content_div)
          print("Row number:",row_number,"Paragraphs from find('p') is EMPTY:")

     
    if row_number % modulo_for_print == 0:
     print(f"Processed row: {row_number}") 
                   
    
    return text_content


In [ ]:
# FUNCTION TO "EFFICIENTLY" CREATE IN "BATCHES" THE TAB-SEPARATED (TSV) FILE
# CONTAINING THE TEXT PASSAGES (AKA KNOWLEDGE BASE OR CONTEXT)

def batches_append_to_tsv(data_generator, output_path, batch_size, modulo_for_print):  

    batch = []       
    row_num = []                

    # chunk_created_passages_list =[extract_text_from_wikipedia_html(document_text, 
    #                                                 text_position_in_chunk+chunk_size*chunk_number, 
    #                                                 modulo_for_print)  for chunk_number, chunk in enumerate(data_generator)
    #                                                                    # ONLY WHEN THE CONDTION BELOW IS MET WILL THE NEXT "FOR" BE ENTERED AND RETURN THE FUNCTION
    #                                                                    #if chunk_number <= 2  # This acts like a break condition                                                         
    #                                                                    for text_position_in_chunk, document_text in enumerate(chunk)                                               
    #                                                                                                                                  ]

    for chunk_number, chunk in enumerate(data_generator):
          
          for text_position_in_chunk, document_text in enumerate(chunk):
                
                chunk_created_passages_list =[extract_text_from_wikipedia_html(document_text, 
                                                                text_position_in_chunk+chunk_size*chunk_number, 
                                                                modulo_for_print)
                                                ]
                
                batch = batch + chunk_created_passages_list  
                row_num.append(text_position_in_chunk+chunk_size*chunk_number)

                     
          
          if len(batch) > batch_size:

            #print(row_num)
            #print(len(batch))
            
            data_created = zip(row_num,batch)
            # Filter out tuples where the second element is missing or empty
            data_created = [tup for tup in data_created if tup[1]]            

            # Convert the list to a DataFrame             
            df_batch = pd.DataFrame(data_created) 
            #df.to_csv(output_path, sep='\t', mode='a', header=not pd.io.common.file_exists(output_path), index=False)
            df_batch.to_csv(output_path, sep='\t', mode='a', header=None, index=False)
            
            # Clear the batch and row numbers after writing to the file
            row_num.clear()
            batch.clear()
            chunk_created_passages_list.clear()
            del data_created
            del df_batch
            gc.collect()     
          
          
    if len(batch) > 0:
         
         #print(row_num)
         #print(len(batch))

         data_created = zip(row_num,batch)
         # Filter out tuples where the second element is missing or empty
         data_created = [tup for tup in data_created if tup[1]]            

         # Convert the list to a DataFrame             
         df_batch = pd.DataFrame(data_created) 
         #df.to_csv(output_path, sep='\t', mode='a', header=not pd.io.common.file_exists(output_path), index=False)
         df_batch.to_csv(output_path, sep='\t', mode='a', header=None, index=False)
      
         # Clear the batch and row numbers after writing to the file
         row_num.clear()
         batch.clear()
         chunk_created_passages_list.clear()
         del data_created
         del df_batch
         gc.collect()     

    return 


In [ ]:
# RUN FUNCTION ABOVE
%%time

#file_path_to_google_nq_dataset = os.path.join(os.getcwd(), "data",'v1.0-simplified_simplified-nq-train.jsonl.gz')
file_path_to_google_nq_dataset = os.path.join(os.getcwd(), "data",'google_nq_simplified_dataset_v1.gz') 
chunk_size =300
tsv_file_path1 = os.path.join(os.getcwd(), "data","text_passages.tsv")
batch_size=4000
modulo_for_print=5000

batches_append_to_tsv( data_generator = chunks_generator_for_gz_zipped_file(data_path = file_path_to_google_nq_dataset, chunk_size = chunk_size),
                       output_path=tsv_file_path1,
                       batch_size=batch_size,
                       modulo_for_print = modulo_for_print
                       
                       )
                       

Processed row: 0
Row number: 3967 Paragraphs from find('p') is EMPTY:
Processed row: 5000
Row number: 6853 Paragraphs from find('p') is EMPTY:
Row number: 7466 Paragraphs from find('p') is EMPTY:
Row number: 8986 Paragraphs from find('p') is EMPTY:
Row number: 9368 Paragraphs from find('p') is EMPTY:
Row number: 9934 Paragraphs from find('p') is EMPTY:
Processed row: 10000
Row number: 12938 Paragraphs from find('p') is EMPTY:
Row number: 13860 Paragraphs from find('p') is EMPTY:
Processed row: 15000
Row number: 15297 Paragraphs from find('p') is EMPTY:
Row number: 15430 Paragraphs from find('p') is EMPTY:
Row number: 16590 Paragraphs from find('p') is EMPTY:
Row number: 19403 Paragraphs from find('p') is EMPTY:
Row number: 19953 Paragraphs from find('p') is EMPTY:
Processed row: 20000
Row number: 21887 Paragraphs from find('p') is EMPTY:
Row number: 22796 Paragraphs from find('p') is EMPTY:
Processed row: 25000
Row number: 25166 Paragraphs from find('p') is EMPTY:
Row number: 29337 Par

In [ ]:
%%time

tsv_file_path1 = os.path.join(os.getcwd(), "data","text_passages.tsv")
data_size = 0
#for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
for chunk in pd.read_csv(tsv_file_path1, sep='\t', names=['id', 'text'], chunksize=100):  
    data_size += len(chunk)


data_size 

CPU times: user 25.6 s, sys: 1.99 s, total: 27.6 s
Wall time: 1min 53s


307216

### III-A.2. EXPLORE BY LOADING CHUNKS FROM SAVED TEXT PASSAGES (IF TOO BIG FOR MEMORY)

In [ ]:
# GENERATOR FUNCTION FOR "CHUNK-LOADING" DATA FROM A FILE (HERE TSV)

def chunks_generator_for_TSV_file_limited(data_path, chunk_size):

    chunks_read= []
    chunk_count = 0 

    # READ ONLY A NUMBER OF ROWS FROM THE SAVED-TO-DISK DATASET THAT IS EQUAL TO "chunk_size"
    # for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
    for chunk in pd.read_csv(data_path, sep='\t', names=['id', 'text'], chunksize=chunk_size):

        chunks_read.append(chunk) 
                  
        # LIMIT THE NUMBER OF CHUNKS TO READ FROM THE SAVED-TO-DISK DATASET
        if chunk_count >= 50:  
                break            
        # YIELD ONLY EXIST WITHIN A FUNCTION
        yield chunk
        chunk_count += 1 
    
    return chunks_read



In [ ]:
chunks_read_df = pd.concat(chunks_generator_for_TSV_file_limited(tsv_file_path1, chunk_size=1), ignore_index=True)
print(chunks_read_df.head())

# RELEASE MEMORY
chunks_read_df = None
del chunks_read_df
gc.collect()

   id                                               text
0   0  Email marketing is the act of sending a commer...
1   1  Tracy McConnell , better known as `` The Mothe...
2   2  Human fertilization is the union of a human eg...
3   3  The following is a list of the top National Fo...
4   4  The Roanoke Colony ( / \u02c8ro\u028a\u0259\u0...


0

### III-A.3. CREATE THE "INPUTS" TENSORS AS HD5 FILE

In [ ]:

# print("\n TRANSFERT of inputs TO Gpu:")
# inputs_tensor = {key: value.to(device) for key, value in inputs.items()}
# print(type(inputs_tensor), len(inputs_tensor))
# inputs_tensor.keys()
# path_to_inputs_tensor =  os.path.join(os.getcwd(), "data","inputs_tensor.pth")

# TO SAVE AS TENSOR WITH torch.save() USE EXTENSION ".pt" (STANDS FOR PYTORCH) WHICH IS TYPICALLY TO SAVE (AKA SERIALIZED) A "SINGLE" OR A COLLECTION OF 
# TENSORS WHILE EXTENSION ".pth" IS TO SAVE MODELS INCLUDING THEIR ARCHITECTURE ALONG THE WEIGHTS
# PyTorch does NOT support APPENDING to a .pt or .pth file. THUS YOU MUST load the existing file, add the new data, and then save it again. 
# "NOT PREFERRED" BECAUSE WILL EVENTUALLY RUN OUT OF MEMORY!!!!
#torch.save(inputs_tensor, path_to_inputs_tensor)


# TO SAVE AS NUMPY
# Convert tensors to numpy arrays and save them
#inputs_numpy = {key: value.cpu().detach().numpy() for key, value in inputs_tensor.items()}

#path_to_inputs_tensor_as_numpy = os.path.join(os.getcwd(), "data","inputs_tensor.npy")

# numpy np.save() does NOT support appending data to an existing .npy file AS np.save() overwrites the file AT EACH CALL
#YOU MUST load the existing file, add the new data, and then save it again.
# "NOT PREFERRED" BECAUSE WILL EVENTUALLY RUN OUT OF MEMORY!!!!
#np.save(path_to_inputs_tensor_as_numpy, inputs_numpy)


# PREFERRED IS TO USE HDF5 (Hierarchical Data Format version 5) FORMAT 
# FOR EFFICIENT INCREMENTAL APPENDING AND SAVING DATA

In [ ]:
def create_inputs_tensor_as_numpy(path_to_df, chunk_size, max_length, batch_size, output_path):
    
    # INSTANTIATE THE pre-trained TOKENIZER TO BE APPLIED THE TEXT PASSAGES (AKA KNOWLEDGE BASE )
    tokenizer_passage_encoder = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')  
                                                                            
    batch_of_inputs = []    

    # for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):                
    for chunk_df in pd.read_csv(path_to_df, sep='\t', names=['id', 'text'], chunksize=chunk_size):          

          # CREATE A LIST WITH AS MANY ELEMENTS AS THE DATAFRAME CHUNK
          passages_list = list(chunk_df['text'])

          #print("\n CPU compute of inputs:") 
          # Hugging Face's DPRQuestionEncoderTokenizer "CANNOT" USE GPU. Tokenization is a CPU-bound process 
          # IT WILL RETURN A DICTIONARY CONTAINING PyTorch TENSORS 
          inputs = tokenizer_passage_encoder(passages_list, return_tensors='pt', padding=True, max_length=max_length, truncation=True)

          # MANY libraries (like NumPy) READ/WRITE EASILY FROM HDF5 FILE AND  NumPy arrays use less memory compared to PyTorch tensors
          inputs_numpy = {key: value.detach().numpy() for key, value in inputs.items()}
          del inputs
          #print(inputs_numpy.keys())

          batch_of_inputs.append(inputs_numpy)

          if len(batch_of_inputs)>batch_size:
               
               print(len(batch_of_inputs))
               
               # "h5py.File(output_path, 'a')"  OPENS THE FILE IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY 
               # CREATE IF "at output_path"
               with h5py.File(output_path, 'a') as f:

                    # Iterate over each key in the first dictionary to initialize datasets
                    for key in batch_of_inputs[0].keys():

                         #print("key when batch size met", key)
                         # Stack all data for the current key across dictionaries into a single NumPy array
                         data_array = np.stack([d[key] for d in batch_of_inputs])

                         # Check if the dataset already exists in the file
                         if key in f:

                              # Resize the dataset to accommodate the new data
                              dataset = f[key]
                              dataset.resize((dataset.shape[0] + data_array.shape[0]), axis=0)
                              dataset[-data_array.shape[0]:] = data_array
                         else:
                              # Create a new dataset with expandable dimensions
                              f.create_dataset(key, data=data_array, maxshape=(None,) + data_array.shape[1:])

               # Clear the batch and row numbers after writing to the file
               passages_list.clear()
               del inputs_numpy
               batch_of_inputs.clear()               
               del data_array
               gc.collect()          
       
    if len(batch_of_inputs)>0: 
                         
          print(len(batch_of_inputs))

          # "h5py.File(output_path, 'a')"  OPENS THE FILE IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY 
          # CREATE IF "at output_path"
          with h5py.File(output_path, 'a') as f:

               # Iterate over each key in the first dictionary to initialize datasets
               for key in batch_of_inputs[0].keys():

                    #print("key when batch size NOT met:", key, )
                    # Stack all data for the current key across dictionaries into a single NumPy array
                    data_array = np.stack([d[key] for d in batch_of_inputs])

                    # Check if the dataset already exists in the file
                    if key in f:

                         # Resize the dataset to accommodate the new data
                         dataset = f[key]
                         dataset.resize((dataset.shape[0] + data_array.shape[0]), axis=0)
                         dataset[-data_array.shape[0]:] = data_array
                    else:
                         # Create a new dataset with expandable dimensions
                         f.create_dataset(key, data=data_array, maxshape=(None,) + data_array.shape[1:])

          # Clear the batch and row numbers after writing to the file
          passages_list.clear()
          del inputs_numpy
          batch_of_inputs.clear()               
          del data_array
          gc.collect() 
                    

In [ ]:
%%time

divisor_value=1688
#tsv_file_path1 = os.path.join(os.getcwd(), "data","text_passages.tsv")
chunk_size_for_inputs_tensor_as_numpy = divisor_value 
max_length = 256
batch_size_for_inputs_tensor_as_numpy = 10 
path_to_inputs_tensor_as_numpy = os.path.join(os.getcwd(), "data"," inputs_tensors_saved_as_numpy.h5")



create_inputs_tensor_as_numpy(path_to_df=tsv_file_path1, 
                                 chunk_size=chunk_size_for_inputs_tensor_as_numpy,
                                 max_length = max_length,
                                 batch_size=batch_size_for_inputs_tensor_as_numpy,
                                 output_path=path_to_inputs_tensor_as_numpy,
                                 )
                            

11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key when batch size met attention_mask
11
key when batch size met input_ids
key when batch size met token_type_ids
key 

In [ ]:
with h5py.File(path_to_inputs_tensor_as_numpy, 'r') as f:
    for key in f.keys():
        dataset = f[key]
        print(f"{key} : {dataset.shape}")

attention_mask : (182, 1688, 256)
input_ids : (182, 1688, 256)
token_type_ids : (182, 1688, 256)


### III-A.4. CREATE EMBEDDINGS FROM INPUTS TENSORS AS HD5 FILE
 * An HDF5 (Hierarchical Data Format version 5) file is a binary file format used to store large amounts of data efficiently and in a STRUCTURED way.

In [ ]:
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [ ]:
def create_text_passage_embeddings(file_source_path, chunks_grabbed_from_source_h5_file, batch_size, output_path):    
                                                                           
    # INSTANTIATE THE pre-trained EMBEDDING MODEL FOR KNOWLEDGE BASE (AKA PASSAGES)
    model_passages_encoder = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')   
    model_passages_encoder = model_passages_encoder.to(device)
    
    batch_of_embeddings = []    

    # Open the HDF5 file
    with h5py.File(file_source_path, 'r') as f:          

    # Iterate over the keys (assuming the keys are 'input_ids', 'attention_mask', etc.)
     for key in f.keys()[0]:

          dataset = f[key]
          total_chunks_in_source_h5_file= dataset.shape[0]
          print(f"\n THESE ARE THE TOTAL ROWS (CHUNKS) IN INPUT HD5 FILE : {total_chunks_in_source_h5_file} with dataset shape {dataset.shape}")
          #print(f"\n THESE ARE THE keys : {f.keys()}")
          # Load data in chunks
          for start in range(0, total_chunks_in_source_h5_file, chunks_grabbed_from_source_h5_file):

               end = min(start + chunks_grabbed_from_source_h5_file, total_chunks_in_source_h5_file)
               #chunk_data = dataset[start:end]
               #print(chunk_data.keys())  
               
               # GRAB INPUTS SAVED AS NUMPY
              
               
               input_ids = f['input_ids'][start:end]             
               attention_mask = f['attention_mask'][start:end]            
               token_type_ids = f['token_type_ids'][start:end]  
               

               # CONVERT INPUTS SAVED AS NUMPY TO TENSORS AND THEN MOVE THEM TO GPU
               inputs = {'input_ids': torch.tensor(input_ids.reshape(-1, max_length)).to(device),
                         'attention_mask': torch.tensor(attention_mask.reshape(-1, max_length)).to(device),
                         'token_type_ids':torch.tensor(token_type_ids.reshape(-1, max_length)).to(device),
                         }
  
               # ON CPU
               # inputs = {'input_ids': torch.tensor(input_ids),
               #           'attention_mask': torch.tensor(attention_mask),
               #           'token_type_ids':torch.tensor(token_type_ids)
               #           }
  

               print("START EMBEDDINGS")
               # Perform the forward pass WITH gradients DISABLED for efficiency
               with torch.no_grad(): 
                    embeddings = model_passages_encoder(**inputs).pooler_output   

               # Convert to NumPy AND Detach and move to CPU 
               embeddings_numpy = embeddings.detach().cpu().numpy()    
               print("SHAPE FOR CHUNK OF EMBEDDED DOCS:", embeddings_numpy.shape)    

               batch_of_embeddings.append(embeddings_numpy)
               print("EMBEDDINGS FINISHED")               
               print(f"LENGTH OF batch_of_embeddings {len(batch_of_embeddings)}")    

               if len(batch_of_embeddings)>batch_size:
                    
                    print(len(batch_of_embeddings))                                  

                    # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
                    # IF IT DOES NOT ALREADY EXIST.                    
                    with h5py.File(output_path, 'a') as f2:

                         # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                         # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                         if "dataset" not in f2:

                              # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                              # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                              # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                              # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                              # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                          
                              maxshape = (None,) + batch_of_embeddings[0].shape[1:]
                              # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                              dset = f2.create_dataset("dataset", shape=(0,) + batch_of_embeddings[0].shape[1:], 
                                                      maxshape=maxshape, 
                                                      # SPECIFY THAT EACH CHUNK APPENDED MUST HAVE AS NUMBER OF ROWS: "batch_of_embeddings[0].shape[0]"
                                                      # CAN BE ANY NUMBER OF YOUR CHOICE. THE SYSTEM INTERNALLY SPLITS THE NUMBER OF ROW RECEIVED TO CREATE
                                                      # THE SPECIFY NUMBER OF ROWS IN EACH CHUNK WITH The HDF5 format ALLOWING partial chunk UNITL MORE DATA
                                                      # COMES IN LATER
                                                      # chunks=((batch_of_embeddings[0].shape[0]),) + \
                                                      chunks=((batch_of_embeddings[0].shape[0])/chunks_grabbed_from_source_h5_file,) + \
                                                     
                                                                                                              batch_of_embeddings[0].shape[1:])

                         else:
                              # Reopen the existing dataset if it already exists
                              dset = f2["dataset"]


                         # COULD DO USE "np.stack([np_arr for np_arr in batch_of_embeddings])" HERE INSTEAD OF THE LOOP BUT NOT SURE MEMORY CAN HANDLE IT!!!!
                         # HOWEVER IT WOULD DO BETTER BECAUSE ONLY A SINGLE "WRITE" OPERATION BUT WE LOOSE THE FLEXIBILITY OF ADDING "SMALLER" CHUNKS IN LATER
                         # RUNS. np.vstack() JUST CONCATENATE ALONG axis 0 (all arrays MUST HAVE the same shape except in this dimension) 
                         # WHILE np.stack() ALSO ADDS A NEW DIMENSION TO THE OUTPUT: INPUT ARRAYS OF (100, 50) GIVE 10, 100, 50).
                         # This WOULD BE useful if you want to keep each INPUT ARRAY AS separate SLICE in the stacked result 
                         # IT WOULD WORK AS FOLLOWS:
                         # stacked_array = np.vstack(batch_of_embeddings)
                         # Resize dataset to accommodate the stacked data
                         #dset.resize((dset.shape[0] + stacked_array.shape[0]), axis=0)
                         # Write the entire stacked array to the end of the dataset
                         #dset[-stacked_array.shape[0]:] = stacked_array
                         for array in batch_of_embeddings:

                              # Resize dataset to accommodate new data
                              dset.resize((dset.shape[0] + array.shape[0]), axis=0)
                              
                              # Write data to the end of the dataset
                              dset[-array.shape[0]:] = array

                    # Clear the batch and row numbers after writing to the file  
                    del embeddings                  
                    del embeddings_numpy
                    batch_of_embeddings.clear()     
                    gc.collect()          
       
    if len(batch_of_embeddings)>0: 
                              
          print(len(batch_of_embeddings))          

          with h5py.File(output_path, 'a') as f2:

               # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
               # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
               if "dataset" not in f2:

                    # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                    # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                    # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                    # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                    # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                             
                    maxshape = (None,) + batch_of_embeddings[0].shape[1:]
                    # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                    dset = f2.create_dataset("dataset", shape=(0,) + batch_of_embeddings[0].shape[1:], 
                                        maxshape=maxshape, 
                                        # SPECIFY THAT EACH CHUNK APPENDED MUST HAVE AS NUMBER OF ROWS: "batch_of_embeddings[0].shape[0]"
                                        # CAN BE ANY NUMBER OF YOUR CHOICE. THE SYSTEM INTERNALLY SPLITS THE NUMBER OF ROW RECEIVED TO CREATE
                                        # THE SPECIFY NUMBER OF ROWS IN EACH CHUNK WITH The HDF5 format ALLOWING partial chunk UNITL MORE DATA
                                        # EVENTUALLY COMES IN LATER
                                        # chunks=((batch_of_embeddings[0].shape[0]),) + \
                                        chunks=((batch_of_embeddings[0].shape[0])/chunks_grabbed_from_source_h5_file,) + \
                                        
                                                                                                              batch_of_embeddings[0].shape[1:])
                    

               else:
                    # # Reopen the existing dataset if it already exists
                    dset  = f2["dataset"]


               for ind, array in enumerate(batch_of_embeddings):

                    print("lAST BATCH: LIST ITEM #:", ind)
                    # Resize dataset to accommodate new data
                    dset.resize((dset.shape[0] + array.shape[0]), axis=0)
                    
                    # Write data to the end of the dataset
                    dset[-array.shape[0]:] = array

          # Clear the batch and row numbers after writing to the file  
          del embeddings        
          del embeddings_numpy
          batch_of_embeddings.clear() 
          gc.collect() 
                    

In [ ]:
%%time

path_to_inputs_tensor_as_numpy = os.path.join(os.getcwd(), "data","inputs_tensors_saved_as_numpy.h5")
divisor_value=1688
max_length=256

file_path_for_text_passage_embeddings = path_to_inputs_tensor_as_numpy
# NUMBER OF CHUNKS IN SOURCE H5 FILE GRABBED AT ONCE
# MUST BE A "DIVISOR" OF data_size/divisor_value:  [1, 2, 7, 13, 14.0, 26.0, 91.0, 182.0]
chunks_grabbed_from_source_h5_file =  2
batch_size=3
output_path_for_text_passage_embeddings = os.path.join(os.getcwd(), "data","text_passage_embeddings.h5")



create_text_passage_embeddings(file_source_path=file_path_for_text_passage_embeddings,
                               chunks_grabbed_from_source_h5_file=chunks_grabbed_from_source_h5_file,
                               batch_size=batch_size,
                               output_path=output_path_for_text_passage_embeddings,
                               )

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



 THESE ARE THE TOTAL ROWS (CHUNKS) IN INPUT HD5 FILE : 182 with dataset shape (182, 1688, 256)
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 1
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 2
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 3
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 4
4
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 1
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 2
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batch_of_embeddings 3
START EMBEDDINGS
SHAPE FOR CHUNK OF EMBEDDED DOCS: (3376, 768)
EMBEDDINGS FINISHED
LENGTH OF batc

In [ ]:
#output_path_for_text_passage_embeddings = os.path.join(os.getcwd(), "data","text_passage_embeddings.h5")

with h5py.File(output_path_for_text_passage_embeddings, 'r') as f:

    for key in f.keys():

        dataset = f[key]
        print(f"{key} : {dataset.shape}")

dataset : (307216, 768)


### III-A.5. CREATE FAISS INDECES FROM TEXT PASSAGES EMBEDDINGS
 

In [ ]:
#device = torch.device('cuda:0')

def create_faiss_IndexIVFFlat_index(file_path_for_embeddings_hd5, indeces_path, chunk_size, divisor, number_cluster_for_embeddings=100, 
                                    sample_embeddings_size=50000):

    # Open the HDF5 file
    with h5py.File(file_path_for_embeddings_hd5, 'r') as f: 

    # Iterate over the keys associated with the different datasets inside the HD5 file
     for key in f.keys():  

        # Grabs iteratively each dataset 
        dataset = f[key]

        # Dimensionality of embeddings
        embeddings_vectors_length = dataset.shape[1]

        # NUMBER OF VECTORS EMBEDDINGS (ROWS)
        total_embeddings = dataset.shape[0]
        
        # IF EMBEDDINGS ARE "NORMALIZED":
        # Initialize a quantizer
        # quantizer = faiss.IndexFlatIP(d) 
        # Initialize the IVF index
        # index_cpu = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_COSINE)  
              
        quantizer = faiss.IndexFlatL2(embeddings_vectors_length) 
        # Initialize the IVF index
        # "number_cluster_for_embeddings" IS THE Number of clusters INTO WHICH that ALL THE EMBEDDINGS IN THE SOURCE FILE will be partitioned.
        index_cpu = faiss.IndexIVFFlat(quantizer, embeddings_vectors_length, number_cluster_for_embeddings, faiss.METRIC_L2)

        # "index.nprobe" IS typically set after the index has been created and before performing a search
        # "index.nprobe" SPECIFIES HOW MANY OF THE CLUSTERS MADE USING THE EMBEDDINGS WILL BE INCLUDED FOR A SEARCH, balancing EFFICIENCY and ACCUARCY.                 
        # BY Setting nprobe equal to nlist the index will search "ALL" clusters in the index TO MAXIMIZE "RECALL".
        index_cpu.nprobe = number_cluster_for_embeddings

        # Transfer CPU index to GPU (device 0)
        try:            
            #print("FAISS-GPU resources are available.")
            gpu_resources = faiss.StandardGpuResources()
            index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)  
        except AttributeError:
            print(f" FAISS-GPU resources are not available. Ensure the GPU version of FAISS is installed.", "\n")
        
        
        # SPECIFY THE SIZE OF EMBEDDINGS SAMPLE TO BE USED FOR "TRAINING" the index
        sample_embeddings = dataset[:sample_embeddings_size]          
        index.train(sample_embeddings)

        # TRANSFER BACK THE INDEX TO CPU BEFORE SAVINGS
        index_cpu = faiss.index_gpu_to_cpu(index)

        # Set index to use disk storage
        # If the index size exceeds available MEMORY, FAISS AUTOMATICALLY WILL USE disk-backed storage.        
        faiss.write_index(index_cpu, indeces_path)

        index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)

        # Add embeddings in chunks
        for start in range(0, total_embeddings, chunk_size):
            end = min(start + chunk_size, total_embeddings)
            embeddings_chunk = np.array(dataset[start:end], dtype='float32')
            # Add to disk-backed index
            index.add(embeddings_chunk)  

            # Periodically save to disk
            # Save every 10 chunks
            if start % (chunk_size * divisor) == 0: 

                index_cpu = faiss.index_gpu_to_cpu(index) 
                # USE "partial_index" AVOIDS fragmentation AND unnecessary I/O on THE "FINAL" file
                faiss.write_index(index_cpu, os.path.join(os.getcwd(),"data","partial_index.faiss"))
                print(f"Partial Index saved at chunk starting at row: {start} and ending at row: {end}")
                index = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)

        index_cpu = faiss.index_gpu_to_cpu(index) 
        # Final save to ensure all embeddings are written
        faiss.write_index(index_cpu, indeces_path)

In [ ]:
output_path_for_text_passage_embeddings = os.path.join(os.getcwd(), "data","text_passage_embeddings.h5")
path_to_save_faiss_indeces= os.path.join(os.getcwd(), "data","disk_index.faiss")
chunks_of_embeddings = int(data_size/4)
divisor = 26
number_cluster_for_embeddings = 2048 #Max possible!!!!)
sample_embeddings_size = data_size

create_faiss_IndexIVFFlat_index(file_path_for_embeddings_hd5=output_path_for_text_passage_embeddings, 
                                indeces_path=path_to_save_faiss_indeces, 
                                chunk_size=chunks_of_embeddings,
                                divisor = divisor,
                                number_cluster_for_embeddings=number_cluster_for_embeddings, 
                                sample_embeddings_size=sample_embeddings_size)

Index saved at chunk 0 to 76804


### III-A.6. RAG MODEL

In [ ]:
%%time

# FIND MAX LENGHTS ACCROSS ALL TEXT PASSAGES
tsv_file_path1 = os.path.join(os.getcwd(), "data","text_passages.tsv")

#for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
for chunk in pd.read_csv(tsv_file_path1, sep='\t', names=['id', 'text'], chunksize=100):  

    passages_max_len = 0
    column_name = chunk.columns[1]
    length_of_texts = [len(row[column_name]) for index, row in chunk.iterrows()]
    if passages_max_len < max(length_of_texts):
        passages_max_len =  max(length_of_texts)

passages_max_len = passages_max_len +10
print(passages_max_len)

132868
CPU times: user 32.2 s, sys: 1.7 s, total: 33.9 s
Wall time: 51.6 s


In [ ]:
def create_folder_containing_text_passages_with_embeddings(text_passages_path, file_path_for_embeddings_hd5, 
                                                           chunk_size, batch_size, max_len, output_path):


    # Open the HDF5 file
    with h5py.File(file_path_for_embeddings_hd5, 'r') as f: 
     

    # Iterate over the keys associated with the different datasets inside the HD5 file
     for key in f.keys():  

        # Grabs iteratively each dataset 
        dataset = f[key]
        total_embeddings = dataset.shape[0]
        #print(total_embeddings)

        batch_of_dicts = []

        #for chunk_df in pd.read_csv(file_path, skiprows=range(1, start_row + 1), chunksize=chunk_size):
        for chunk in pd.read_csv(text_passages_path, sep='\t', names=['id', 'text'], chunksize=chunk_size): 
            
            column_name = chunk.columns[1]
            
            list_of_dicts_from_chunks = [{column_name: row[column_name], "title": 'passage '+ str(index), "embeddings": dataset[index]} 
                                                                                                             for index, row in chunk.iterrows()
                                        ] 
            
            batch_of_dicts = batch_of_dicts + list_of_dicts_from_chunks
            #print(len(batch_of_dicts))         
              

            if len(batch_of_dicts)> batch_size:
                        
                print(len(batch_of_dicts)) 

                # 'S100' is for a fixed-length ASCII bytes USED FOR STRINGS (HERE 100 BYTES)
                # ('f4', (chunk_size, 768)) is for  N-D numpy arrays with shape (chunk_size, 768) of float32 ('f4') type
                vector_sizes = batch_of_dicts[0]["embeddings"].shape[0]
                #print(vector_sizes)

                dtype = np.dtype([(column_name, 'S' + str(max_len)),  ('title', 'S' + str(30)),  ('embeddings', ('f4', (1, vector_sizes))) ])

                #structured_array = np.array([tuple(d.values()) for d in batch_of_dicts], dtype=dtype)
                structured_array = np.array([(d[column_name].encode('utf-8'), d["title"].encode('utf-8'), d["embeddings"]) 
                                                                                                                      for d in batch_of_dicts], dtype=dtype)
                             

                # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
                # IF IT DOES NOT ALREADY EXIST.                    
                with h5py.File(output_path, 'a') as f2:

                        # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                        # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                        if "dataset" not in f2:

                            # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                            # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                            # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                            # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                            # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                        
                            maxshape = (None,) + structured_array.shape[1:]
                            
                            # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                            dset = f2.create_dataset("dataset", data=structured_array, maxshape=maxshape)

                            # Add column names as metadata
                            dset.attrs['column_names'] = [column_name, 'title', 'embeddings']
                                                  
                        else:
                            # Reopen the existing dataset if it already exists
                            dset = f2["dataset"]
                            dset.resize((dset.shape[0] + len(batch_of_dicts)), axis=0)
                            dset[-len(batch_of_dicts):] = structured_array


                # Clear the batch and row numbers after writing to the file         
                list_of_dicts_from_chunks.clear()
                batch_of_dicts.clear()    
                del structured_array 
                gc.collect()          

    if len(batch_of_dicts)>0: 
                            
        print(len(batch_of_dicts))       

        # 'S100' is for a fixed-length ASCII bytes USED FOR STRINGS (HERE 100 BYTES)
        # ('f4', (chunk_size, 768)) is for  N-D numpy arrays with shape (chunk_size, 768) of float32 ('f4') type
        vector_sizes = batch_of_dicts[0]["embeddings"].shape[0]
        #print(vector_sizes)

        dtype = np.dtype([(column_name, 'S' + str(max_len)),  ('title', 'S' + str(30)),  ('embeddings', ('f4', (1, vector_sizes))) ])
        
        #structured_array = np.array([tuple(d.values()) for d in batch_of_dicts], dtype=dtype)
        structured_array = np.array([(d[column_name].encode('utf-8'), d["title"].encode('utf-8'), d["embeddings"]) 
                                                                                                                      for d in batch_of_dicts], dtype=dtype)

        # "h5py.File(output_path, 'a')" OPENS A IN  "append" mode WITHOUT DELETING EXISTING data OR AUTOMATICALLY CREATE IT "at output_path"
        # IF IT DOES NOT ALREADY EXIST.                    
        with h5py.File(output_path, 'a') as f2:

                # AN HD5 (Hierarchical Data Format version 5) FILE CAN CONTAIN MULTIPLE DATASETS. WE CALL "dataset" THE ONE WE ARE CREATING
                # Check if the dataset already exists OTHERWISE creates it with expandable dimensions
                if "dataset" not in f2:

                    # WHEN INITIALIZING "dataset" ALLOW UNLIMITED ROWS VIA "None" (HERE EACH ROW WILL BE A CHUNK). "dataset" WILL INCREMENTALLY 
                    # RECEIVE EMBEDDINGS OF DOCUMENTS IN CHUNKS OF NUMPY ARRAYS HAVING EACH THE NUMBER OF "COLUMNS" DEFINED BY: 
                    # batch_of_embeddings[0].shape[1:] WHERE "batch_of_embeddings[0]" IS THE FIRST ARRAY IN THE LIST "batch_of_embeddings" AND
                    # ".shape[1:]" IS THE SHAPE OF THE THAT ARRAY EXCEPT THE FIRST DIMENSION. THUS IT IS JUST THE NUMBER OF "COLUMNS"
                    # NOTE: WE SKIPPED GRABBING THE FIRST DIMENSION (# OF ROWS) BECAUSE WE ALREADY ADDRESSED THAT VIA "NONE"
                                                
                    maxshape = (None,) + structured_array.shape[1:]
                    
                    # CREATE WITH 0 ROWS AND "batch_of_embeddings[0].shape[1:]" NUMBER OF COLUMNS
                    dset = f2.create_dataset("dataset", data=structured_array, maxshape=maxshape)

                    # Add column names as metadata
                    dset.attrs['column_names'] = [column_name, 'title', 'embeddings']
                                            
                else:
                    # Reopen the existing dataset if it already exists
                    dset = f2["dataset"]
                    dset.resize((dset.shape[0] + len(batch_of_dicts)), axis=0)
                    dset[-len(batch_of_dicts):] = structured_array


        # Clear the batch and row numbers after writing to the file         
        list_of_dicts_from_chunks.clear()
        batch_of_dicts.clear()    
        del structured_array 
        gc.collect()          

In [ ]:
output_path_for_text_passage_embeddings = os.path.join(os.getcwd(), "data","text_passage_embeddings.h5")
chunk_size_for_text_passages_with_embeddings = int(data_size/364)
batch_size_for_text_passages_with_embeddings = chunk_size_for_text_passages_with_embeddings*4

output_path_for_text_passage_with_embeddings_added = os.path.join("/mnt/h/RAG","text_passages_with_embeddings_added.h5")
#output_path_for_text_passage_with_embeddings_added = os.path.join('/home/koffi', 'text_passages_with_embeddings_added.h5')

create_folder_containing_text_passages_with_embeddings(tsv_file_path1, output_path_for_text_passage_embeddings, 
                                                       chunk_size=chunk_size_for_text_passages_with_embeddings,
                                                       batch_size=batch_size_for_text_passages_with_embeddings,
                                                       max_len = passages_max_len,
                                                       output_path=output_path_for_text_passage_with_embeddings_added)

In [92]:
from datasets import Dataset, concatenate_datasets

def create_dataset_in_chunks(hdf5_file_path, chunk_size, batch_size, output_path_for_dataset_chunks):

    with h5py.File(hdf5_file_path, 'r') as f:
        
        
        # Get the dtype of the dataset
        dtype =  f["dataset"].dtype          

        total_size = f["dataset"].shape[0]  
        #print(total_size)  

        chunk_num = 0
        for start in range(0, total_size, chunk_size):
                              
               end = min(start + chunk_size, total_size)
               
               # Read a batch of data
               data_chunk = f["dataset"][start:end]

               chunk_num +=1
            
               list_of_dicts_from_chunk = [] 
               # Convert the batch to a list of dictionaries
               #list_chunk = [{"data": item} for item in data_chunk]
               for item in data_chunk:
                    
                    text = item[dtype.names[0]].decode('utf-8')
                    title = item[dtype.names[1]].decode('utf-8')
                    embeddings = item[dtype.names[2]]

                    row_into_dict = {"text":text, "title":title, "embeddings":embeddings}

                    list_of_dicts_from_chunk.append(row_into_dict)
                    #print(len(list_of_dicts_from_chunk))
                    

               if len(list_of_dicts_from_chunk) >batch_size:
                    
                    print(len(list_of_dicts_from_chunk))
                    
                    # Create a Dataset from the chunk
                    chunk_dataset = Dataset.from_list(list_of_dicts_from_chunk)
                         
                    # Convert the Dataset to a pandas DataFrame by using the 'to_pandas()' method
                    #chunk_df = chunk_dataset.to_pandas() 

                    # SAVE AS PARQUET FILE
                    #chunk_df.to_parquet( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}.parquet'), index=False) 
                    chunk_dataset.save_to_disk( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}'))
                    #chunk_dataset.save_to_disk( output_path_for_dataset_chunks)   
               
               list_of_dicts_from_chunk.clear()
               chunk_dataset = None
               chunk_df = None
               del chunk_dataset, chunk_df
               gc.collect
          
        if len(list_of_dicts_from_chunk) >0:
                    
                    print(len(list_of_dicts_from_chunk))
                    
                    # Create a Dataset from the chunk
                    chunk_dataset = Dataset.from_list(list_of_dicts_from_chunk)
                         
                    # Convert the Dataset to a pandas DataFrame by using the 'to_pandas()' method
                    #chunk_df = chunk_dataset.to_pandas() 

                    # SAVE AS PARQUET FILE
                    #chunk_df.to_parquet( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}.parquet'), index=False)  
                    chunk_dataset.save_to_disk( os.path.join(output_path_for_dataset_chunks, f'chunk_{chunk_num}'))  
                    #chunk_dataset.save_to_disk( output_path_for_dataset_chunks)  

                    
               
        list_of_dicts_from_chunk.clear()
        chunk_dataset = None
        chunk_df = None
        del chunk_dataset, chunk_df
        gc.collect

                              
                    
output_folder_for_dataset_chunks = os.path.join("/mnt/h/RAG/passages")
create_dataset_in_chunks(output_path_for_text_passage_with_embeddings_added, chunk_size=43888,batch_size=43887, 
                         output_path_for_dataset_chunks = output_folder_for_dataset_chunks)
            

43888


Saving the dataset (0/2 shards):   0%|          | 0/43888 [00:00<?, ? examples/s]

43888


Saving the dataset (0/2 shards):   0%|          | 0/43888 [00:00<?, ? examples/s]

43888


...

Saving the dataset (0/2 shards):   0%|          | 0/43888 [00:00<?, ? examples/s]

43888


Saving the dataset (0/2 shards):   0%|          | 0/43888 [00:00<?, ? examples/s]

43888


Saving the dataset (0/2 shards):   0%|          | 0/43888 [00:00<?, ? examples/s]

In [99]:
from datasets import load_from_disk, concatenate_datasets
chunk_dirs = [f"/mnt/h/RAG/passages/chunk_{i+1}" for i in range(7)]
datasets = [load_from_disk(chunk_dir) for chunk_dir in chunk_dirs]
# Concatenate all chunks into a single dataset
combined_dataset = concatenate_datasets(datasets)

combined_dataset.save_to_disk(output_folder_for_dataset_chunks)

Saving the dataset (0/13 shards):   0%|          | 0/307216 [00:00<?, ? examples/s]

In [ ]:
#not needed: CONVERT FILE WITH PASSAGES TO "PICKLE" FORMAT!!!!!!!!
#pkl_file_path = os.path.join(os.getcwd(), "data","passages.pkl")
# Save the list of dictionaries to a .pkl file
#with open(pkl_file_path, 'wb') as f:
#    pickle.dump(passages_as_list_of_dict, f)

# Check if the file exists
#if not os.path.exists(pkl_file_path):
#    raise FileNotFoundError(f"The file {pkl_file_path} does not exist.")


# INSTANTIATE THE "RAG" the TOKENIZER
tokenizer_for_retriever = RagTokenizer.from_pretrained("facebook/rag-sequence-nq")

# Example question
#question = "What is machine learning?"
#question = "What is the capital Deutscheland?"
question = "Who won the world series in 2020?"
#question = "what is Berlin?"

# Tokenize the input
input_ids = tokenizer_for_retriever(question, return_tensors="pt").input_ids



# INSTANTIATE the RAG model
model_result_generation = RagTokenForGeneration.from_pretrained("facebook/rag-sequence-nq")


# Get hidden states for the question USING "question_encoder" 
# ALSO Disable gradient calculation for inference
with torch.no_grad():  
    question_hidden_states = model_result_generation.question_encoder(input_ids)[0] 


# Convert TENSOR to NumPy array BY MOVING IT to CPU and convert to NumPy array (cpu FRAMEWORK)
question_hidden_states_np = question_hidden_states.cpu().numpy()

output_folder_for_dataset_chunks = os.path.join("/mnt/h/RAG/passages")
path_to_save_faiss_indeces= os.path.join(os.getcwd(), "data","disk_index.faiss")

retriever = RagRetriever.from_pretrained("facebook/rag-sequence-nq",
                                         index_name="custom", output_retrieved=False,
                                         tokenizer = tokenizer_for_retriever,
                                         passages_path=output_folder_for_dataset_chunks, 
                                         index_path=path_to_save_faiss_indeces
                                         )


#  Use the retriever to retrieve relevant documents
docs_dict = retriever(input_ids, question_hidden_states=question_hidden_states_np, return_tensors="pt")

# RETRIEVE EMBEDDINGS OF DOCUMENTS (Assuming you have retrieved embeddings for the documents)
retrieved_doc_embeds = docs_dict['retrieved_doc_embeds']  # Shape (n_docs, embed_dim)

# GET CONTEXT INPUT IDs
n_docs =docs_dict['context_input_ids'].shape[0]
batch_size = input_ids.shape[0]
context_input_ids = docs_dict['context_input_ids'].reshape(batch_size * n_docs, -1)


if 'context_attention_mask' not in docs_dict:
    docs_dict['context_attention_mask'] = torch.ones_like(context_input_ids)
#print(docs_dict['context_attention_mask'].shape)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

### III-A.6.1. RESPONSE BASED ON COSINE SIMILARITY WITH RETRIEVED DOCUMENTS

In [ ]:
# Expand question_hidden_states to [1, 5, 1, 768]
expanded_question_hidden_states = question_hidden_states.unsqueeze(1).unsqueeze(2).expand(-1, retrieved_doc_embeds.size(1), retrieved_doc_embeds.size(2), -1)

# Compute cosine similarity along the last dimension
doc_scores_cosine = F.cosine_similarity(expanded_question_hidden_states, retrieved_doc_embeds, dim=-1)

# Squeeze out any unnecessary singleton dimensions for further processing
doc_scores_cosine = doc_scores_cosine.squeeze(-1)  # Shape should now be [1, 5]


# Generate the answer conditioned on retrieved documents
outputs_cosine = model_result_generation.generate(
    input_ids=input_ids,
    context_input_ids=context_input_ids,
    doc_scores=doc_scores_cosine,
    context_attention_mask=docs_dict['context_attention_mask']
)



# Decode the generated output
generated_text_cosine = tokenizer_for_retriever.batch_decode(outputs_cosine, skip_special_tokens=True)[0]
print(generated_text_cosine)


 los angeles , california


### III-A.6.2. RESPONSE BASED ON EUCLIDEAN DISTANCE WITH RETRIEVED DOCUMENTS

In [ ]:
# Ensure that retrieved_doc_embeds is squeezed along the third dimension (singleton) before cdist
doc_scores_euclidean = torch.cdist(question_hidden_states.unsqueeze(1), retrieved_doc_embeds.squeeze(2), p=2)  # Shape: [1, 5]

# Remove the extra singleton dimension if present to get shape [1, 5]
doc_scores_euclidean = doc_scores_euclidean.squeeze(1) 



# Generate the answer conditioned on retrieved documents
outputs_euclidean = model_result_generation.generate(
                         input_ids=input_ids, 
                         context_input_ids=context_input_ids, 
                         doc_scores=doc_scores_euclidean,
                         context_attention_mask = docs_dict['context_attention_mask']
                         )


# Decode the generated output
generated_text_euclidean = tokenizer_for_retriever.batch_decode(outputs_euclidean, skip_special_tokens=True)[0]
print(generated_text_euclidean)


 los angeles dodgers


### III-A.6.3. RESPONSE BASED ON DOT PRODUCT WITH RETRIEVED DOCUMENTS

In [ ]:
# Ensure the shapes are correct before batch matrix multiplication
question_hidden_states_unsq = question_hidden_states.unsqueeze(1) # Shape: (batch_size, 1, embed_dim)

retrieved_doc_embeds_squeezed = retrieved_doc_embeds.squeeze(2)  # Shape: (batch_size, n_docs, embed_dim)


# Perform batch matrix multiplication
doc_scores_dotprod = torch.bmm(question_hidden_states_unsq, retrieved_doc_embeds_squeezed.transpose(1, 2)).squeeze(1)  # Shape: (batch_size, n_docs)



# Generate the answer conditioned on retrieved documents
outputs_dotprod = model_result_generation.generate(
                         input_ids=input_ids, 
                         context_input_ids=context_input_ids, 
                         doc_scores=doc_scores_dotprod,
                         context_attention_mask = docs_dict['context_attention_mask']
                         )


# Decode the generated output
generated_text_dotprod = tokenizer_for_retriever.batch_decode(outputs_dotprod, skip_special_tokens=True)[0]
print(generated_text_dotprod)



 los angeles , california


# X- APPENDIX

In [ ]:
# TO LOAD DATA IN BATCHES FROM AN HD5 FILE
import h5py
from torch.utils.data import Dataset, DataLoader

class HDF5Dataset(Dataset):
    def __init__(self, file_path):
        self.file = h5py.File(file_path, 'r')
        self.data = self.file['dataset']  # Adjust to your dataset's key

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx]  # Load the data item by index

dataset = HDF5Dataset('your_file.h5')
dataloader = DataLoader(dataset, batch_size=32)

for data in dataloader:
    # Process your data
    pass


In [ ]:
def load_embeddings_in_chunks(h5_file_path, dataset_name, chunk_size):
    """Generator to yield embeddings in chunks from HDF5 file."""
    
    with h5py.File(h5_file_path, 'r') as f:
        dataset = f[dataset_name]
        total_embeddings = dataset.shape[0]
        embedding_dim = dataset.shape[1]

        for start in range(0, total_embeddings, chunk_size):
            end = min(start + chunk_size, total_embeddings)
            embeddings_chunk = dataset[start:end]
            yield embeddings_chunk




def add_embeddings_to_index(index, h5_file_path, dataset_name, chunk_size):
    """Load embeddings in chunks and add them to the FAISS index."""
    for embeddings_chunk in load_embeddings_in_chunks(h5_file_path, dataset_name, chunk_size):
        embeddings_chunk = np.array(embeddings_chunk, dtype='float32')  # FAISS requires float32
        index.add(embeddings_chunk)  # Add chunk to the FAISS index

def search_faiss_index(index, query_vectors, k=5):
    """Search the FAISS index for the k-nearest neighbors of the query vectors."""
    query_vectors = np.array(query_vectors, dtype='float32')
    distances, indices = index.search(query_vectors, k)
    return distances, indices

# Parameters
h5_file_path = 'path/to/your/embeddings.h5'
dataset_name = 'your_dataset_name'  # Dataset within the HDF5 file containing embeddings
embedding_dim = 768  # Adjust based on your embeddings' dimensionality
chunk_size = 10000  # Adjust chunk size based on available memory
index_type = "Flat"  # Or use "IVF" for larger datasets
k = 5  # Number of nearest neighbors to retrieve in search

# Initialize and populate FAISS index
index = create_faiss_index(embedding_dim, index_type)
add_embeddings_to_index(index, h5_file_path, dataset_name, chunk_size)

# If using an IVF index, train it with a sample of data before adding embeddings
if index_type == "IVF" and not index.is_trained:
    with h5py.File(h5_file_path, 'r') as f:
        training_data = f[dataset_name][:chunk_size]  # Use first chunk for training
    index.train(training_data.astype('float32'))

# Example search with random query vectors (replace with your own queries)
query_vectors = np.random.rand(10, embedding_dim).astype('float32')  # Replace with actual queries
distances, indices = search_faiss_index(index, query_vectors, k)

# Output results
print("Nearest neighbors' indices:", indices)
print("Nearest neighbors' distances:", distances)
